Qualitative Dashboard

1. Set your userkey to access the correct data, and update the database host if necessary.
2. Run the cells
3. Review the threat scenarios.

This is a read only notebook, nothing in this notebook will affect the data stored in the database.

The Overall Likelihood and Risk Level tables are derived from NIST SP-800-30


In [ ]:
userkey = "140"
dbhost  = 'localhost'

In [ ]:
import pandas as pd
import ipywidgets as widgets
import psycopg2
import qual

conn = psycopg2.connect("postgres://csc452:csc452@" + dbhost + "/csc452")  
cursor = conn.cursor()  
cursor.execute('SET mydomain.userkey = ' + userkey)

triages = pd.read_sql_query ('SELECT name, description, vulnerability, pervasive, impact, impact_name, impact_description, source_name, source_description, capacity, init FROM triagev', conn)

for i in triages.index:
    v = qual.qdict[qual.norm(triages.at[i,'vulnerability'])]
    l = qual.qdict[qual.norm(triages.at[i,'init'])]
    im = qual.qdict[qual.norm(triages.at[i,'impact'])]
    triages.at[i,'vuln'] = v
    triages.at[i,'imp'] = im
    triages.at[i,'cap'] = qual.qdict[qual.norm(triages.at[i,'capacity'])]
    triages.at[i,'likely'] = l
    
    # look up the overall likelihood of the attack succeeding
    o = qual.overall[l][v]
    triages.at[i,'overall_likelihood'] = o
    
    # look up the qualitative risk level
    triages.at[i,'risk_level'] = qual.risk_level[o][im]

print ("Summary of Threat Scenarios")
    
triages = triages.set_index ('name')

# filter out newly added events
tr = triages.loc[triages['source_name']!='source name']
tr.filter (items=['name', 'description', 'vuln', 'imp', 'cap', 'overall_likelihood', 'risk_level'])

In [ ]:
print ("Summary of Threat Sources")
tr.filter (items=['name', 'source_name', 'source_description'])